## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [129]:
import pandas as pd
client_pay_ability = pd.read_csv('/datasets/data.csv')
display(client_pay_ability)
client_pay_ability.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Всего в таблице 12 столбцов, при этом количество значений в столбцах 'days_employed', 'total_income' различное, значит, есть 'пропущенные значения'. Данные значения могут быть пропущены как случайно, так и не случайно, возможно, клиент не предоставил информацию по данным пунктам намеренно.
Пять столбцов с типом данных - 'строка', еще пять - 'целые числа', оставшиеся два - 'вещественные числа'. Это отрицательные числа в днях трудового стажа (необходимо уточнить у коллег из кредитного отдела, возможно, это перерерывы в стаже или маркировка выхода на пенсию) и дроби, показывающие ежемесячный доход. Встречаются строки разного регистра и категории, дублированные по смыслу ('purpose'), это может быть неудобным для фильтрации данных.
Каждая строка таблицы позволяет узнать как о семейном, материальном и социальном положении клиента, так и о его финансовых целях.
Необходимо удалить 'пропущенные значения', 'дубликаты'. Для решения задачи будут важны столбцы 'children', 'family_status', 'debt'.

Описание данных
•	children — количество детей в семье
•	days_employed — общий трудовой стаж в днях
•	dob_years — возраст клиента в годах
•	education — уровень образования клиента
•	education_id — идентификатор уровня образования
•	family_status — семейное положение
•	family_status_id — идентификатор семейного положения
•	gender — пол клиента
•	income_type — тип занятости
•	debt — имел ли задолженность по возврату кредитов
•	total_income — ежемесячный доход
•	purpose — цель получения кредита


## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 2. Предобработка данных

### Обработка пропусков


In [132]:
#display(client_pay_ability.isnull().sum())
#display(client_pay_ability.groupby('income_type') ['education'].count())
employee_avg = client_pay_ability[client_pay_ability['income_type'] == 'сотрудник']['total_income'].median()
client_pay_ability.loc[client_pay_ability['income_type'] == 'сотрудник', 'total_income'] = client_pay_ability.loc[client_pay_ability['income_type'] == 'сотрудник', 'total_income'].fillna(employee_avg)

companion_avg = client_pay_ability[client_pay_ability['income_type'] == 'компаньон']['total_income'].median()
#display(companion_avg)
client_pay_ability.loc[client_pay_ability['income_type'] == 'компаньон', 'total_income'] = client_pay_ability.loc[client_pay_ability['income_type'] == 'компаньон', 'total_income'].fillna(companion_avg)

state_service_avg = client_pay_ability[client_pay_ability['income_type'] == 'госслужащий']['total_income'].median()
#display(state_service_avg)
client_pay_ability.loc[client_pay_ability['income_type'] == 'госслужащий', 'total_income'] = client_pay_ability.loc[client_pay_ability['income_type'] == 'госслужащий', 'total_income'].fillna(state_service_avg)

pensioner_avg = client_pay_ability[client_pay_ability['income_type'] == 'пенсионер']['total_income'].median()
#display(pensioner_avg)
client_pay_ability.loc[client_pay_ability['income_type'] == 'пенсионер', 'total_income'] = client_pay_ability.loc[client_pay_ability['income_type'] == 'пенсионер', 'total_income'].fillna(pensioner_avg)

businessman_avg = client_pay_ability[client_pay_ability['income_type'] == 'предприниматель']['total_income'].median()
#display(businessman_avg)
client_pay_ability.loc[client_pay_ability['income_type'] == 'предприниматель', 'total_income'] = client_pay_ability.loc[client_pay_ability['income_type'] == 'предприниматель', 'total_income'].fillna(businessman_avg)
#display(client_pay_ability.groupby('income_type')['total_income'].count())
display(client_pay_ability.isnull().sum())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

In [131]:
#display(client_pay_ability.groupby('income_type') ['education'].count())


def age_group(dob_years):
    
    if dob_years < 17:
                return 'детский возраст'
        
    if dob_years >= 18 and dob_years <= 44:
                return 'молодой возраст'
    
    if dob_years >= 45 and dob_years <= 59:
                return 'средний возраст'
    return 'пожилой возраст'
             
client_pay_ability['age_group'] = client_pay_ability['dob_years'].apply(age_group)
#display(client_pay_ability['age_group'].value_counts())
#display(client_pay_ability.groupby('age_group')['days_employed'].count())

junior_avg = client_pay_ability[client_pay_ability['age_group'] == 'детский возраст']['days_employed'].median()
#display(junior_avg)
client_pay_ability.loc[client_pay_ability['age_group'] == 'детский возраст', 'days_employed'] = client_pay_ability.loc[client_pay_ability['age_group'] == 'детский возраст', 'days_employed'].fillna(junior_avg)

young_avg = client_pay_ability[client_pay_ability['age_group'] == 'молодой возраст']['days_employed'].median()
#display(young_avg)
client_pay_ability.loc[client_pay_ability['age_group'] == 'молодой возраст', 'days_employed'] = client_pay_ability.loc[client_pay_ability['age_group'] == 'молодой возраст', 'days_employed'].fillna(young_avg)

middle_avg = client_pay_ability[client_pay_ability['age_group'] == 'средний возраст']['days_employed'].median()
#display(middle_avg)
client_pay_ability.loc[client_pay_ability['age_group'] == 'средний возраст', 'days_employed'] = client_pay_ability.loc[client_pay_ability['age_group'] == 'средний возраст', 'days_employed'].fillna(middle_avg)

old_avg = client_pay_ability[client_pay_ability['age_group'] == 'пожилой возраст']['days_employed'].median()
#display(old_avg)
client_pay_ability.loc[client_pay_ability['age_group'] == 'пожилой возраст', 'days_employed'] = client_pay_ability.loc[client_pay_ability['age_group'] == 'пожилой возраст', 'days_employed'].fillna(old_avg)

display(client_pay_ability.isnull().sum())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

### Вывод

Проверим таблицу с помощью двух последовательных методов: поиск на наличие пропусков и их суммирование. Как мы и предполагали, пропущены значения в столбцах 'days_employed' и 'total_income', клиенты банка могут об этом не сообщать, но для кредитного отдела - это архиважная информация. Необходимо заменить пропущенные данные по доходу медианным значением категории типа источника дохода. Определяем по столбцу, в котором нет пропусков, сколько людей у нас в каждой категории по типу дохода. Пропуски есть в категориях:'госслужащий, компаньон, пенсионер, предприниматель, сотрудник'. По каждой из них находим медианный доход и заменяем им пропуски. У нас всего два 'предпринимателя', один из них без указания дохода - в данном случае, и медианное, и среднеарифметическое значения будут одинаковы, мы приравняем доход второму, т.к. доля таких клиентов в общей массе незаметна.

Выясняем, что пропуски в колонке 'days_employed' в тех же 5 категориях. Заменим пропущенные значения медианным значением по возрастной группе. С помощью функции создаем отдельный столбец 'age_group'.
Найдем медианные значения по возрастным группам, которые классифицируем согласно ВОЗ:
детский возраст - 0-17
молодой возраст - 18-44
средний возраст - 45-59
пожилой возраст - 60+
Все пропущенные значения заменены.




### Замена типа данных

In [133]:
client_pay_ability['days_employed'] = client_pay_ability['days_employed'].astype('int')
client_pay_ability['total_income'] = client_pay_ability['total_income'].astype('int')
client_pay_ability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

Отрицательные дробные числа в столбце 'days_employed' и положительные дробные числа в столбце 'total_income' выглядят громоздко и могут помешать корректной работе методов, поэтому изменим их тип данных с 'float' на 'integer'. Проверяем, что у нас получилось. Теперь у нас только два типа данных. Будет некорректно заменять отрицательные числа на положительные, т.к.это нарушит логику данной таблицы.

### Обработка дубликатов

In [69]:
client_pay_ability.duplicated().sum()
client_pay_ability.drop_duplicates(inplace = True)
client_pay_ability['education'] = client_pay_ability['education'].str.lower()
#print(client_pay_ability['education'].value_counts())
client_pay_ability['family_status'] = client_pay_ability['family_status'].str.lower()
#print(client_pay_ability['family_status'].value_counts())
#print(client_pay_ability['income_type'].value_counts())
#print(client_pay_ability['purpose'].value_counts())
client_pay_ability.duplicated().sum()

0

### Вывод


Мы обнаружили, что у нас 54 дубликата. Удаляем их, но,возможно, еще есть дубликаты по смыслу - выявляем вручную. Скорее всего, они встречаются в столбцах с типом данных 'строка'. Обработаем вручную каждый такой столбец на предмет уникальных значений.
1. В столбце 'education' очень много дубликатов из-за регистра. Переводим весь столбец в нижний регистр.
2. В столбце 'family_status' дубликатов нет, но одна из категорий начинается с заглавной буквы. Переводим весь столбец в нижний регистр.
3. В столбце 'income_type' все в порядке.
4. В столбце 'purpose' слишком много разных словоформ и словосочетаний с одинаковым смыслом. Для этого столбца выполним лемматизацию и категоризацию.
Проверяем еще раз - дубликатов нет.

### Лемматизация

In [148]:
#display(client_pay_ability['purpose'].unique())

from pymystem3 import Mystem
m = Mystem()
purpose_category = client_pay_ability['purpose'].unique()

lemmas = []
for i in purpose_category: 
        result = ' '.join(m.lemmatize(i)).strip()
        lemmas.append(result)
    
#display(lemmas)
from collections import Counter
#display(Counter(lemmas))

def purpose_count(row):
    purpose_category = m.lemmatize(row['purpose'])
    
    if 'свадьба' in purpose_category:
        return 'организация свадьбы'

    if 'жилье'in purpose_category:
        return 'приобретение недвижимости'
    
    if 'образование' in purpose_category:
        return 'получение образования'
    
    if 'автомобиль' in purpose_category:
        return 'приобретение автомобиля'
    
    if 'недвижимость' in purpose_category:
        return 'приобретение недвижимости'
    
    
client_pay_ability['credit_purpose'] = client_pay_ability.apply(purpose_count, axis=1)
#display(client_pay_ability.head(10))

display(client_pay_ability['credit_purpose'].value_counts())

приобретение недвижимости    10840
приобретение автомобиля       4315
получение образования         4022
организация свадьбы           2348
Name: credit_purpose, dtype: int64

### Вывод

Необходимо определить все уникальные значения столбца 'purpose' и провести их лемматизацию.
Импортируем pymystem3, специальную библиотеку для русского языка  и прописываем цикл для лемматизации каждого уникального значения. Опредляем 4 категории целей кредита. Полученные леммы соотносим с уникальным значением в столбце 'purpose', создавая новый столбец 'credit_purpose'с чистой целью кредита. Суммируем все значения по категориям и выясняем, что самая популярная цель кредита 'приобретение недвижимости'.


### Категоризация данных

In [149]:

def children_count(row):
    
    children = row['children']
    
    if children < 1:
        return 'нет детей'

    if children >= 1:
        return 'есть дети'

        
client_pay_ability['children_id'] = client_pay_ability.apply(children_count, axis=1)
#print(client_pay_ability.head(10))

def family_count(row):
    
    family_status_id = row['family_status_id']
    
    if family_status_id < 1:
        return 'в браке'

    if family_status_id >= 1:
        return 'не в браке'

        
client_pay_ability['family_id'] = client_pay_ability.apply(family_count, axis=1)
print(client_pay_ability.head(10))
                    
   


   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   Среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   СРЕДНЕЕ             1   
8         2          -6929         35    ВЫСШЕЕ             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [151]:
#print(client_pay_ability['total_income'].mean())
#print(client_pay_ability['total_income'].median())
#print(client_pay_ability['total_income'].max())
#print(client_pay_ability['total_income'].min())
a = client_pay_ability['total_income']
r = pd.qcut(a, 5)
display(r)

def income_count(row):
    
    total_income = row['total_income']
    
    if total_income <= 98662:
        return 'бедные'

    if total_income >= 98663 and total_income <= 132141:
        return 'уязвимые'
    
    if total_income >= 132142 and total_income <= 161151:
        return 'средний класс'
 
    if total_income >= 161152 and total_income <= 214269:
        return 'обеспеченные'
    
    if total_income > 214270:
        return 'состоятельные'
    
client_pay_ability['income_group'] = client_pay_ability.apply(income_count, axis=1)
#print(client_pay_ability.head(10))
print(client_pay_ability.groupby('debt')['income_group'].value_counts())

0        (214269.6, 2265604.0]
1          (98661.6, 132141.8]
2         (132141.8, 161151.2]
3        (214269.6, 2265604.0]
4         (132141.8, 161151.2]
                 ...          
21520    (214269.6, 2265604.0]
21521     (132141.8, 161151.2]
21522     (20666.999, 98661.6]
21523    (214269.6, 2265604.0]
21524     (20666.999, 98661.6]
Name: total_income, Length: 21525, dtype: category
Categories (5, interval[float64]): [(20666.999, 98661.6] < (98661.6, 132141.8] < (132141.8, 161151.2] < (161151.2, 214269.6] < (214269.6, 2265604.0]]

debt  income_group 
0     состоятельные    4003
      бедные           3960
      обеспеченные     3945
      уязвимые         3945
      средний класс    3931
1     средний класс     374
      обеспеченные      360
      уязвимые          360
      бедные            345
      состоятельные     302
Name: income_group, dtype: int64


### Вывод

Вернемся к данным столбцов 'children', 'family_status', 'debt', 'total_income'. Необходимо понять, влияет ли семейное положение и наличие детей на платежеспособность. Проведем категоризацию по типу данных:

1. Создадим функцию для разделения данных на категории 'есть дети' и 'нет детей'. Поскольку столбец 'children' содержит количество детей и затруднит подсчет данных. Добавим отдельный столбец 'children_id' только из двух значений.
2. Также поступим с семейным положением. В таблице 'семейный статус' несколько значений, но нас интересует, 'в браке' клиент или 'нет'. Добавим через функцию отдельный столбец 'family_id'.  
3. Также категоризируем столбец 'total_income'. Нам необходимо выбрать верный подход к разделению по доходам. Мы  можем разбить столбец дохода по квантилям данных (метод qcut) - на пять 20%-групп с диапазонными значениями.Для классификации данных групп используем терминологию метода разделения слоев общества по медианному доходу, где (бедные получают = 0,5 от медианы дохода, уязвимые = 0.5-0,75, средний класс - 0,75-2, обеспеченные - 2-4, состоятельные - 4).

 


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
print(client_pay_ability.groupby('debt')['children_id'].value_counts())

debt  children_id
0     нет детей      13090
      есть дети       6640
1     нет детей       1064
      есть дети        677
Name: children_id, dtype: int64


### Вывод

Считаем количество имеющих и не имеющих долги по обеим категориям. Мы видим, что только третья часть всех клиентов имеет детей. В каждой категории рассчитываем процент тех, кто имеет задолженность по кредитам: в категории "нет детей" таких клиентов 8%, в категории "есть дети" их больше  - 10%. Из этого следует, что наличие детей незначительно влияет на возврат кредита, т.е. если у клиента есть дети, риск возникновения задолженности по оплате кредита немного возрастает.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
print(client_pay_ability.groupby('debt')['family_id'].value_counts())

debt  family_id 
0     в браке       11413
      не в браке     8317
1     в браке         931
      не в браке      810
Name: family_id, dtype: int64


In [ ]:
client_pay_ability.groupby('family_id').agg({'debt': ['sum', 'count', 'mean']})

### Вывод

Суммируя общее количество клиентов по статусам 'в браке' и 'не в браке', определяем, что 58% клиентов банка состоит в браке. При этом только 8,1% клиентов из категории 'в браке' не возвращает платежи по кредиту в срок. Чуть больше - 9,7%  - таких клиентов есть в категории 'не в браке'. Семейное положение влияет на возврат кредита, но не в значительной степени. Если клиент не женат/не замужем, риск несвоевременного возврата может быть выше, чем у клиента, который женат/замужем. Хотя это очевидно, в первом случае кредит может брать семья, т.е. два человека - в таком случае кредит вернуть легче, т.к. семейный доход может быть дифференцированным. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [194]:
income_pivot = client_pay_ability.pivot_table(index=['income_group'], columns='debt', values='family_status_id', aggfunc='sum')
income_pivot['ratio'] = income_pivot[0] / income_pivot[1]
income_pivot['total_credit'] = income_pivot[0] + income_pivot[1]
display(income_pivot.sort_values(by = 'income_group', ascending=False))

debt,0,1,ratio,total_credit
income_group,,,,
уязвимые,3992,395,10.106329,4387
средний класс,3786,389,9.732648,4175
состоятельные,3759,322,11.673913,4081
обеспеченные,3719,434,8.569124,4153
бедные,3813,325,11.732308,4138


### Вывод

Согласно полученным данным чаще всего берут кредиты категория'уязвимые' слои населения, реже всего клиенты из категории 'бедные'. Тем не менее, у них больше всего проблем с оплатой кредитов - 11,7%.  Лучше всего возвращают кредиты 'обеспеченные' - 8,6%. У 'состоятельных' медианное значение по общему количеству кредитов, при этом они также плохо возвращают кредиты как и 'бедные' - 11.7%.
Зависимость между уровнем дохода и возвратом кредита не прямопропорциональна, т.к. и бедные, и богатые люди не возвращают их вовремя с одинаковой частотой, возможно, по разным причинам. Наименее рискованными социальными группами для выдачи кредита являются обеспеченные и средний класс (9,7%). Клиенты из категории 'уязвимые' с учетом наибольшого количества кредитов в данном банке входят в среднюю группу риска, хотя процент кредитной задолженности составляет 10,1%. Высокая группа риска - две крайних категории - 'бедные' и 'состоятельные'.


- Как разные цели кредита влияют на его возврат в срок?

In [190]:
data_pivot = client_pay_ability.pivot_table(index=['credit_purpose'], columns='debt', values='family_status_id', aggfunc='sum')
data_pivot['ratio'] = data_pivot[0] / data_pivot[1]
display(data_pivot.sort_values(by = 'credit_purpose', ascending=False))



debt,0,1,ratio
credit_purpose,,,
приобретение недвижимости,9685,808,11.986386
приобретение автомобиля,3783,482,7.848548
получение образования,3439,389,8.840617
организация свадьбы,2162,186,11.623656


### Вывод

Согласно анализу целей кредита чаще всего берут кредиты на 'приобретение недвижимости', поэтому их  чаще и возвращают, и задерживают. На втором месте по частоте получения и задержки кредита - 'приобретение автомобиля'.'Получение образования' и организация свадьбы' - менее популярные цели кредита по понятным причинам: образование - долговременная инвестиция, а свадьба входит в статью расходов на развлечения и редко окупает себя. Таким образом, цель кредита влияет на срок его возврата. Самые высокие риски по кредитам, выдаваемым на недвижимость (11,9% проблемных из общего числа в этой категории) и на свадьбы (11,6%). Меньше риска возникает при выдаче кредита на автомобиль - 7,8%. У кредитов на образование средние риски - 8,8%.


### Шаг 4. Общий вывод

В целях постановленной задачи анализа предоставленных данных. Мы исследовали статистику о платёжеспособности клиентов, предварительно проведя предобработку данных, результатом которой является:
1. Обнаружение случайных и неслучайных пропущенных значений и их замена медианными значением в показателях 'days_employed', 'total_income'.
2. Замена типов данных на целые числа, при этом обнаружены отрицательные значения в трудовом стаже клиентов 'days_employed'. Возможно, это счетчик выхода на пенсию или некорректно преобразованные данные.
3. Выявление и удаление 54 дубликатов, в том числе значения верхнего регистра переведены в нижний для удобства фильтрации. Благодаря проведению лемматизации столбца 'purpose' c множеством значений, осложняющих фильтрацию данных, определены 4 основных категории целей кредита.
4. Проведение категоризации данных  в  столбцах 'family_status_id', 'children', 'total_income' с целью определения категорий семейного и материального положения клиентов.
5. Анализ обработанных данных позволяет сделать следующие выводы:
- наличие детей влиет на сроки возврата кредита, т.к. только третья часть всех клиентов банка имеет детей и 10% должников находится в этой категории. В категории "нет детей" таких клиентов 8%. Из этого следует, что клиенты, не имеющие детей, приоритетнее для выдачи кредита.
- семейное положение влияет на возврат кредита, но не в значительной степени: 58% клиентов банка состоит в браке, при этом только 8,1% клиентов из категории 'в браке' не возвращает платежи по кредиту в срок. 9,7% - таких клиентов есть в категории 'не в браке'. Поэтому более предпочтительны клиенты, состоящие в браке.
- уровень дохода влияет на сроки возврата кредита, но не прямопропорционально, т.к. в группу риска по невозвратным кредитам входят и 'бедные', и 'состоятельные', не возвращая 11,7% кредитов. Процент кредитной задолженности 'уязвимых' слоев населения достаточно высок (10,1%), но это наиболее частые потребители кредитов. Наиболее интересными для кредитования являются 'обеспеченные' c самым низким процентом невозврата (8,6%) и 'средний класс' со средним - 9,7% и высокой потребностью в данном банковском продукте.
- цель кредита влияет на срок его возврата. Самые высокие риски возникают при выдаче денег на 'недвижимость' и 'организацию свадьбы' с задолженностью - 11,9% и 11,6%, соответственно. Хотя 'недвижимость' - наиболее часто встречающаяся цель кредита. У кредитов на образование меньше риска - 8,8%. Своевременный возврат кредитов наиболее вероятен при выдаче кредита на приобретение автомобиля (наименьшая задолженность из всез целей - 7,8%).

Поставленная задача выполнена успешна, результаты данного исследования могут быть учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.